# ⚡ Level 5: Transfer Learning & Pre-trained Models

In this level, you'll learn how to use **pre-trained models** to save time, compute, and data—by leveraging models trained on massive datasets like **ImageNet** or **large language corpora**.

---

## ✔️ Fine-tuning vs Feature Extraction

| Strategy            | What It Means                                               | When to Use                                           |
|---------------------|------------------------------------------------------------|------------------------------------------------------|
| **Feature Extraction** | Freeze pre-trained layers, only train new classifier layers. | When you have **small datasets** or need speed.       |
| **Fine-Tuning**       | Unfreeze part or all of the model and **train everything**. | When you have **more data** or need high accuracy.    |

✅ You can mix both: freeze early layers, fine-tune later ones.

---

## ✔️ Using Pretrained Models for Vision (`torchvision`)

PyTorch makes it easy to load popular pre-trained models for **image classification**, **object detection**, etc.

### Example: ResNet Feature Extraction


In [15]:
import torch
import torchvision.models as models
import torch.nn as nn

# Use the new weights API
from torchvision.models import ResNet18_Weights

# Load pre-trained ResNet18 with default weights
model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head (example: binary classification)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)


✅ You can now train only the new classifier layer while using the powerful pre-trained features.

## ✔️ Using Pretrained Models for NLP (transformers)
- For Natural Language Processing (NLP), use the transformers library 

Example: Sentiment Classification with BERT

In [33]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import Adam

# ✔️ Load tokenizer and pre-trained BERT model for classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# ✔️ Sample texts and binary labels (1 = positive, 0 = negative)
texts = ["I love this movie!", "This was the worst experience ever."]
labels = torch.tensor([1, 0])  # Ground truth

# ✔️ Tokenize the texts
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# ✔️ Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}
labels = labels.to(device)

# ✔️ Set up optimizer
optimizer = Adam(model.parameters(), lr=5e-5)

# ✔️ Training loop (one epoch example)
model.train()
outputs = model(**inputs, labels=labels)

loss = outputs.loss
logits = outputs.logits

loss.backward()
optimizer.step()

print("Loss:", loss.item())
print("Logits:", logits)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loss: 0.5340299606323242
Logits: tensor([[ 0.1685,  0.3404],
        [ 0.2842, -0.2611]], grad_fn=<AddmmBackward0>)


✅ This works for tasks like sentiment analysis, question answering, and more.

## ✅ Summary Table

| Concept                  | PyTorch Example                                                        |
|--------------------------|-------------------------------------------------------------------------|
| Feature Extraction        | Freeze layers: `param.requires_grad = False`                           |
| Fine-Tuning               | Unfreeze layers: `param.requires_grad = True`                          |
| Pre-trained Vision Models | `torchvision.models.resnet18(pretrained=True)`                         |
| Pre-trained NLP Models    | `transformers.BertForSequenceClassification.from_pretrained()`         |
